
# NMR Exercise - Titration of CBM14 with GlcNAc3

## Requirements
Before starting this exercise, you will need to do the following:
* Download and execute CARA: http://www.cara.nmr-software.org/downloads/ (for Windows, version 1.5.5 is the most stable)
* Download and unzip the NMR data: https://github.com/gcourtade/teaching/raw/master/BioCat-C3-Struct-Bio-Tech-2020/CBM14-titration-spectra.zip

## Introduction

This is a Jupyter Notebook, and it is a way to run Python code in an interactive way. To execute the code in each cell, just press click `►|Run` on the toolbar above or use the keyboard: `Shift + Enter`. 

In this exercise you will learn how to calculate the dissociation constant ($K_d$) for a ligand-protein complex, from a series of NMR spectra (15N-HSQC) recorded at different ligand concentrations.

**The protein:** Carbohydrate-Binding Module 14 (CBM14) from Human macrophage chitotriosidase (PDB ID: 6SO0) [1,2]

**The ligand:** Chitotriose, a chitin trisaccharide (GlcNAc3)

![image](images/prot-lig2.png)


### Go through all the steps below and answer all questions that are prompted. Compile your answers in a document and send it to us.

By the end of the exercise you should be able to answer the following questions when analyzing protein-ligand interactions by NMR.
* How can we extract chemical shift perturbations (dependent varialbe) from a titration series (independent variable)?
* Which amino acids take part in ligand interaction? 
* How can I calculate a $K_d$ for a two-site interaction?
* How can we check if the $K_d$ we obtained is valid?


## Theoretical background

#### Kd from chemical shift perturbations
We assume a a two-state process where a ligand ($L$) and protein ($P$) interact to form a complex ($PL$). <br>


![image](images/2state.png)


If we are at $[L]=0 mM$, $[P]$=total protein concentration and $[PL]=0 mM$ and measure the chemical shift of a peak in the 15N-HSQC spectrum, its coordinates would be ($\delta$$H_{0mM}$, $\delta$$N_{0mM}$).

If we add some ligand $[L]=0.5 mM$ and we are on a fast-exchange regime, the chemical shift of the peak will change to new coordinates ($\delta$$H_{0.5mM}$, $\delta$$N_{0.5mM}$).

![image](images/csp2.png)

Now we can calculate the combined chemical shift perturbation of the peak at $L=0.5 mM$ by the using Pythagoras equation:
## $\Delta \delta_{comb} = \sqrt{(\Delta \delta H)^2 + (\frac{\Delta \delta N}{R_{scale}})^2}$

where $\Delta \delta H = \delta H_{0.5mM} - \delta H_{0mM}$, $\Delta \delta N = \delta N_{0.5mM} - \delta N_{0mM}$ and $R_{scale}=6.5$ [3]. At each titration point, we repeat this process to obtain $\Delta \delta_{comb}$ for each $[L]$.

Then we can fit the following equation to the data by varying the parameters $K_d$ and $Q_{max}$ (equal to $\Delta \delta_{comb}$ at saturation).

## $\Delta \delta_{comb}  = Q_{max} \frac{ [P] + [L] + K_d \pm \sqrt{([P]+[L]+K_d)^2 -4[P][L]} }{2[P]}$

See https://www.pnas.org/content/pnas/suppl/2012/10/25/1208822109.DCSupplemental/pnas.201208822SI.pdf if you are interested in how this equation is derived.

## Step 1 - Getting started


First we will open CARA and load a project where we have a finished sequence-specific chemical shift assignment and 15N-HSQC spectra for our titration series.

1. Open CARA.


2. `File > Open` and choose the file `cbm14_GlcNAc3.cara` <br>


![image.png](images/cara1.png)


3. If you get the following warning message, click `Yes`, browse to the location of the spectrum and click `Open`. <br>


![image.png](images/cara9.png)


4. Click on `Systems` and on the `Ass` column to see the sorted NMR assignments for the protein. You can browse through the list to get an idea of what an assignment looks like.<br>


![image.png](images/cara2.png)


5. Click on `Spectra` to see a list of the available spectra. As you see here, we only have 15N-HSQCs, each one for every $[L]$ in the titration.

| experiment | $[L]$ |
|------|---|
|15N-HSQC_106 |   0 mM |
|15N-HSQC_115  |  1.0 mM |
|15N-HSQC_124  |  6.3 mM |
|15N-HSQC_130  |  17.1 mM |
|15N-HSQC_133  |  25.0 mM |

6. Go ahead and open `15N-HSQC_106` by right-clicking on it and choosing `Open Polyscope...`


![image.png](images/cara4.png)


![image.png](images/cara5.png)




7. Whoa, that looks messy! We need to reduce the contour levels. You can do this by clicking `Plane > Set Contour Parameters...`. Set the parameters `Factor: 1.4, Threshold: 45000 and Option:Both` and click `OK`.


8. Much better, you can also play around with different `Threshold` and `Factor` values to see how they affect the appearence of the spectrum. Now we can zoom in a region by simulatenously holding down `Ctrl+Shift` while clicking-and-dragging with the left-button of the mouse. If you click one of the peaks you will see the 1D $^{15}N$ projection as the y-axis and the 1D $^1 H$ projection as the x-axis. In the status bar below, you can read off the corresponding chemical shifts for both dimensions. You will notice the small crosses `+` on the peaks. This should the center of the peak and (since this chemical shift assignment has already been done for us) they are labeled with the atoms in the amino acid that give rise to this signal.


![image.png](images/cara6.png)


9. If we zoom even more, we can see the contours of the peaks. This is very useful for detecting small changes in chemical shifts, as we will do later. If you want to move around you can use minimap in the bottom-eft corner of the window or use the keyboard: `Ctrl+Arrow keys` move the region a small distance, `Page Up` and `Page Down` move up and down by one expansion, `Ctrl+Page Up` and `Ctrl+Page Down` move left and right by one expansion. You can change the size of the zoomed area by using `Ctrl+Shift+Arrow keys`. This will enlarge or shrink your selected window vertically or horizontally.To zoom out again, right-click somewhere on the spectrum and select `Fit Window` or use the keyboard shortcut `Ctrl+Home`.


![image.png](images/cara7.png)

Make sure you are comfortable with moving around, zooming in and out of spectra and changing contour levels before proceeding to Step 2.

## Step 2 - Analyzing spectra

Now it's time to put those CARA-skills to work. 

1. Go to `Plane > Overlay > Add Overlay Layer...` and choose 15N-HSQC_133 (the spectrum at the highest [L]).



2. You may need to adjust the `Threshold` as you did in Step 1.6. Both spectra have the same color and this makes it hard to see differences. Change the color of the new spectrum by going to `Plane > Overlay > Set Positive Color...` and choosing a color that gives a good constrast between the spectra and between the spectra and the background.



3. Explore the spectra overlay by zooming in and out and moving around. In which peaks do you observe the highest changes? Which CBM14 amino acids interact with GlcNAc3?

4. Where in the protein do we find these amino acids? (include a figure in your answer) Does the location of the amino acids make sense? Why? You can visualize the NMR structure of CBM14 (PDB IB: 6SO0) here https://www.rcsb.org/3d-view/6SO0. Alternatively, you can use your preferred molecular visualization software to investigate the structure. 

5. In the exercise we are going to use the side-chain **HD21/HD2 of N52** as a reporter of chemical shift perturbations. To get rid of the overlay, close the spectra display window (PolyScope) by using the menu `File > Close`.


6. Reopen the first spectrum `15N-HSQC_106` by repeating Step 1.5. Adjust contours by setting `Factor: 1.1` and `Threshold:400000` or `Threshold:300000`. Zoom in to the corresponding region, place the curor in the center of the peak using the green $^1H$ and $^{15}N$ 1D projections to help you choose the center, and read off the chemical shifts from the status bar. 

![image.png](images/cara8.png)

7. Fill in the chemical shifts in the corresponding list in the cell below. Repeat the process for the other four spectra. When you have filled in all the shifts, execute the code in the cell by clicking `►|Run` on the toolbar above or press `Shift+Enter`.

In [1]:
# Fill in the chemical shifts below, using commas to separate each entry.
# Do not change the L list

dH = [] # ppm ; 1H shifts from 15N-HSQC spectra
dN = [] # ppm ; 15N shifts from 15N-HSQC spectra

L = [0.0, 1.0,6.3,17.1,25.0] # mM; ligand concentrations

## Step 3 - Fitting data to the model

Now that we have analyzed all the spectra, it is time to fit the data to our model and calculate a $K_d$ for the interaction.
Everything has been prepared for you in the code below. Take a moment go through it and when you are ready, execute the code in the cell by clicking `►|Run` on the toolbar above or press `Shift+Enter`.


In [ ]:
# first we import some additional packages
import matplotlib.pyplot as plt
import numpy as np 
from scipy.optimize import curve_fit

# Protein concentration in mM, constant
P = 0.2 

# we define our two-state binding model
def two_state(L,Kd,Qmax):
    return Qmax* (P+L+Kd - ((P+L+Kd)**2 - 4*P*L)**0.5 )/(2*P)

# we need to convert lists to arrays for this to work
L = np.array(L) 
dH = np.array(dH) 
dN = np.array(dN) 

# calculate the combined changed in chemical shift
dcomb = ((dH - dH[0])**2 + ((dN - dN[0])/6.5)**2)**0.5

# calculate the combined error in chemical shift
dcomb_error = ((0.002)**2 + ((0.02)/6.5)**2)**0.5

# do least squares fitting of our data to the two-state model
# the parameter standard deviation errors are estimated from the diagonal elements of the covariance matrix
popt, pcov = curve_fit(two_state,L,dcomb)
perr = np.sqrt(np.diag(pcov))

# plot the data and the resulting fit
plt.errorbar(L,dcomb,color='k',yerr=dcomb_error,fmt='o',capsize=2,label='data')
modelx = np.linspace(L.min(),L.max(),num=50)
plt.plot(modelx,two_state(modelx,*popt),color='r', label='model')
plt.xlabel('[GlcNAc3] (mM)', fontsize=14)
plt.ylabel(r'$\Delta \delta_{comb} (ppm)$', fontsize=14)
kd = r'{:.1f} $\pm$ {:.1f}'.format(popt[0],perr[0])
qmax = r'{:.3f} $\pm$ {:.3f}'.format(popt[1],perr[1])
text = '$K_d$=' + kd + ' mM \n$Q_{max}$=' + qmax + ' ppm'
plt.text(10,0.005,text,fontsize=12)
plt.legend()
plt.show()

Include the plotted figure as part of your answer. Compare the value you ontained with the literature value (it can be found in the **References** at the end of this document).

## Step 4 - Error estimation 
Is the $K_d$ estimate good? Curve fitting only reports on the least-squares fit to our particular data set. We only have one measurement per $[L]$, so random errors can be reduced by repeating measurements (or by repeating the process with another peak in the 15N-HSQC spectrum). Systematic errors may be reduced by removing their soruce from the measurement (more accurate measurement of $[L]$, adjusted NMR acquision protocol, etc) [4].

Another issue is bias in our dataset. Some points may be more important in constraining the fit (especially where the x-axis distribution is not uniform). We can evaluate the significance of these points to the reported error by using different statistical resampling methods [4]. In this example, we will use jackknifing, which works like this:

1. Create new data set by systematically removing each measurement from the dataset, while leaving the rest.
2. Fit to function.
3. Take average value and standard deviation for the fitted parameters.


Execute the code in the next cell by clicking `►|Run` on the toolbar above or press `Shift+Enter`.

In [ ]:
# Jackknife error estimation

allidx = np.arange(len(L))
parameters = []

for i in allidx:
    this = np.delete(allidx,i)
    nL = np.take(L,this)
    ndcomb = np.take(dcomb,this)
    popt, pcov = curve_fit(two_state,nL,ndcomb)
    parameters.append(popt)
    
    plt.errorbar(nL,ndcomb,color='k',yerr=dcomb_error,fmt='o',capsize=2,label='data')
    modelx = np.linspace(L.min(),L.max(),num=50)
    plt.plot(modelx,two_state(modelx,*popt),color='r', label='model')
    plt.title('Removed [L]={} mM'.format(L[i]))
    plt.xlabel('[GlcNAc3] (mM)', fontsize=14)
    plt.ylabel(r'$\Delta \delta_{comb} (ppm)$', fontsize=14)
    kd = '{:.1f}'.format(popt[0])
    qmax = '{:.4f}'.format(popt[1])
    text = '$K_d$=' + kd + ' mM \n$Q_{max}$=' + qmax + ' ppm'
    plt.text(15,0.005,text,fontsize=12)
    plt.legend()
    plt.show()
    

kd = 'Kd = {:.1f} +/- {:.1f} mM'.format(np.average(np.array(parameters)[:,0]),np.std(np.array(parameters)[:,0]))
qmax = 'Qmax = {:.3f} +/- {:.3f} ppm'.format(np.average(np.array(parameters)[:,1]),np.std(np.array(parameters)[:,1]))

print(kd)
print(qmax)

Compare these estimates and errors with those estimated from the fit using all five measurements. Pay attention to the the individual variation in the parameter estimation for each iteration of the jackknife method. What can you conclude from this?

### Closing remarks

By the end of this exercise you should have learned how to extract chemical shift information from 15N-HSQC spectra and how to study a protein-ligand interaction. Moreover, you have learned about the two-state binding model and how to estimate errors from models fitted to data.

We hope you've enjoyed it and do not hesitate to contact us (now or in the future) if you have any questions.

-Gaston gaston.courtade@ntnu.no <br>
-Finn finn.l.acchmann@ntnu.no

### References
[1] Crasson O., Courtade G., Léonard R. R., Aachmann F. L., Legrand F., Parente R., Baurain D., Galleni M., Sørlie M.,
Vandevenne M. (2017) *Human chitotriosidase: catalytic domain or carbohydrate binding module, who’s leading HCHT’s biological function*. Scientific Reports. 17:2768-2777. https://doi.org10.1038/s41598-017-02382-z<br><br>
[2] Madland E., Crasson O., Vandevenne M., Sørlie M., Aachmann F.L. (2019) *NMR and Fluorescence Spectroscopies Reveal the Preorganized Binding Site in Family 14 Carbohydrate-Binding Module from Human Chitotriosidase*. ACS Omega. 26: 21975-21984. https://doi.org/10.1021/acsomega.9b03043<br><br>
[3] Mulder F. A. A., Schipper D., Bott R., Boelens R. (1999) *Altered flexibility in the substrate-binding site of related native and engineered high-alkaline Bacillus subtilisins*. Journal of Molecular Biology. 292:111-123. https://doi.org/10.1006/jmbi.1999.3034<br><br>
[4] Chris Johnson & Stephen McLaughlin (2019) *Data Fitting, Errors and Binding Curves* Biophysics - MRC Laboratory of Molecular Biology https://www2.mrc-lmb.cam.ac.uk/download/lectures/biophysics/2019/Data_Fitting_and_Errors_ChrisJohnson_StephenMcLaughlin_2019.pdf